# Basic tutorial: image data
#### Author: Matteo Caorsi

This short tutorial shows you how to use the basic functioning of *giotto-deep* API for an imagee classfication task.

Image classifcation is about associating a picture with a label: for example, you would like your model to be able to associate a label `cat` to the image of a cat and a label `dog`to an image of a dog:

![img](./images/image_class.png)

It is important to clarify that the set of labels has to be pre-defined and the modls have to be trained with a sufficient amount of data per label.

The main steps of the tutorial are the following:
 1. creation of a dataset
 2. creation of a model
 3. define metrics and losses
 4. run benchmarks
 5. visualise results interactively

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import numpy as np
import torch
from torch import nn
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.tensorboard.writer import SummaryWriter
import torchvision.models as models
from gtda.diagrams import BettiCurve
from gtda.plotting import plot_betti_surfaces

from gdeep.data.datasets import DatasetBuilder, DataLoaderBuilder
from gdeep.models import FFNet
from gdeep.visualisation import persistence_diagrams_of_activations
from gdeep.data.preprocessors import ToTensorImage
from gdeep.trainer import Trainer


# Initialize the tensorboard writer

In order to analyse the results of your models, you need to start tensorboard.
On the terminal, move inside the `/examples` folder. There run the following command:

```
tensorboard --logdir=runs
```

Then go [here](http://localhost:6006/) after the training of your model to see all the visualisation results.

In [ ]:
writer = SummaryWriter()


# Create your dataset

In this example we will be using the `CIFAR10` dataset, which is a dataset of many thousands of images with ten classes.

In giotto-deep it is enough to call `DatasetBuilder(name="CIFAR10")` to get the dataset!

In [ ]:
db = DatasetBuilder(name="CIFAR10")
ds_tr, ds_val, ds_ts = db.build()
NUMBER_OF_CLASSES = 10


## Preprocessing

The preprocessing steps consist in transformin the imagees to tensors. We have already provided a simple class to do such preprocessing: `ToTensorImage`.

The transormation is computed on the fly, hence bring very light on the memory.

In [ ]:
# Preprocessing


transformation = ToTensorImage((32, 32))
transformation.fit_to_dataset(
    ds_tr
)  # this is useless for this transformation, but in general this is the API

transformed_ds_tr = transformation.attach_transform_to_dataset(ds_tr)
transformed_ds_val = transformation.attach_transform_to_dataset(ds_val)
transformed_ds_ts = transformation.attach_transform_to_dataset(ds_ts)

# use only 320 images from cifar10 for training
train_indices = list(range(32 * 10))
val_indices = list(range(32 * 5))
test_indices = list(range(32 * 5))
dl_tr, dl_val, dl_ts = DataLoaderBuilder(
    (transformed_ds_tr, transformed_ds_val, transformed_ds_ts)
).build(
    (
        {"batch_size": 32, "sampler": SubsetRandomSampler(train_indices)},
        {"batch_size": 32, "sampler": SubsetRandomSampler(val_indices)},
        {"batch_size": 32, "sampler": SubsetRandomSampler(test_indices)},
    )
)


## Define and train your model

In the next cell we show how simple it is to take a model from `torch`, like [ResNet18](https://pytorch.org/vision/main/models/generated/torchvision.models.resnet18.html) and use it for training in Giotto-deep.

In [ ]:
# wrap a sequential model in a torch nn.Module
class model3(nn.Module):
    def __init__(self):
        super(model3, self).__init__()
        self.seqmodel = nn.Sequential(
            models.resnet18(pretrained=True), nn.Linear(1000, NUMBER_OF_CLASSES)
        )

    def forward(self, X):
        return self.seqmodel(X)


model = model3()


## Training the model

We have prepard te dataset, we have prepared the model... it is now tim to train it!

In giotto-deep it is just a matter of initialising the `Trainer` and run it with `train`. All the outputs, the exceptions, cross validation... everything you is just a parameter for the trainer. Have a look at the next cell!

In [ ]:
from torch.optim import SGD

loss_fn = nn.CrossEntropyLoss()

pipe = Trainer(model, (dl_tr, dl_ts), loss_fn, writer)

# train the model
pipe.train(
    SGD,
    3,
    False,
    {"lr": 0.01},
    {"batch_size": 32, "sampler": SubsetRandomSampler(train_indices)},
)


# Evaluate model

In the next section we compute the confusion matrix on the entire training dataloader.

In [ ]:
pipe.evaluate_classification(NUMBER_OF_CLASSES)


# Simply use interpretability tools

After th training we would like to inspect what the model has learned. This is possible thanks to interpretability tools.

Here below we show you how simple it is, in giotto-deep, to use many interpretabnility tools in one single line of code.

The example below uses the `GuidedGradCAM` algorithm: given an input `datum`, the modl computes the gradients via backpropagation till the specified layder `conv1` and then averages over teh chanels of the convolutional layer.

The result is then displayed as superimposition of the initial image, with a red/green hue to identify which pixes are pushing for and against the classification of teh given image into the selected `class`.

In [ ]:
from gdeep.analysis.interpretability import Interpreter
from gdeep.visualisation import Visualiser

inter = Interpreter(pipe.model, method="GuidedGradCam")
datum = next(iter(dl_tr))[0][0].reshape(1, 3, 32, 32)
layer = pipe.model.seqmodel[0].layer2[0].conv1
class_ = 1
output = inter.interpret_image(datum, class_, layer)

# visualise the interpreter
vs = Visualiser(pipe)
try:
    vs.plot_interpreter_image(inter)
except AssertionError:
    print("The heatmap is made of all zeros...")


# Appendix

More advanced stuff ahead!

## Extract inner data from your models

For deeper analysis of the working of deep learning models, it is possible, in giotto-deep, to easily extract tons of parameters from your models:
Here below we show you how to extract:
 - each layer parameter,
 - the decision boundary,
 - the activation functions given an input datum,
 - the gradients at each node, given an input and a target.

In [ ]:
from gdeep.models import ModelExtractor

me = ModelExtractor(pipe.model, loss_fn)

list_of_layers = me.get_layers_param()

for k, item in list_of_layers.items():
    print(k, item.shape)


In [ ]:
# the decision boundary will be available on tensorboard, in the projectors section.
x = next(iter(dl_tr))[0][0]
if x.dtype is not torch.int64:  # cannot backpropagate on integers!
    res = me.get_decision_boundary(x, n_epochs=1)
    res.shape


In [ ]:
x = next(iter(dl_tr))[0]
list_activations = me.get_activations(x)
len(list_activations)


In [ ]:
x, target = next(iter(dl_tr))  # a whole batch!
if x.dtype is torch.float:  # cannot backpropagate on integers!
    for gradient in me.get_gradients(x, target=target)[1]:
        print(gradient.shape)


## Visualise activations and other topological aspects of your model

Finally, we show you how it is possible to extract, layer per layer, given a certain input batch, the persistence diagrams computed on the activation space. This means that, for a fixed layer, the input batch is representated as a point cloud in the space of activations for that selected layer. Out of this point cloud we compute the persistence diagrams. In giotto-deep this is literally one line of code.

In [ ]:
# the diagrams can be seen on tensorboard!
vs.plot_persistence_diagrams(x)
